In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 53.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ab9ad63cd868b4558f356a1d2be87a887ea5012f69eaa59bab1aeacccde8b837
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import *
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import *
spark=SparkSession.builder.appName('practise').getOrCreate()

In [ ]:
emp = [(1,"Smith",-1,"2018","10","M",3000), 
    (2,"Rose",1,"2010","20","M",4000), 
    (3,"Williams",1,"2010","10","M",1000), 
    (4,"Jones",2,"2005","10","F",2000), 
    (5,"Brown",2,"2010","40","",-1), 
    (6,"Brown",2,"2010","50","" ,-1) 
  ]

In [ ]:
empColumns = ["emp_id","name","superior_emp_id","year_joined", "dept_id","gender","salary"]

In [ ]:
empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show()

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-------+------+------+
|emp_id|    name|superior_emp_id|year_joined|dept_id|gender|salary|
+------+--------+---------------+-----------+-------+------+------+
|     1|   Smith|             -1|       2018|     10|     M|  3000|
|     2|    Rose|              1|       2010|     20|     M|  4000|
|     3|Williams|              1|       2010|     10|     M|  1000|
|     4|   Jones|              2|       2005|     10|     F|  2000|
|     5|   Brown|              2|       2010|     40|      |    -1|
|     6|   Brown|              2|       2010|     50|      |    -1|
+------+--------+---------------+-----------+-------+------+------+



In [ ]:
empDF.filter("gender !=''").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
+------+--------+---------------+-----------+-----------+------+------+



In [ ]:
dept = [("Finance",10,3000),("Marketing",20,2000),("Sales",30,5000),("IT",40,6000)]
deptColumns = ["dept_name","dept_id","salary"]

In [ ]:
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)
 |-- salary: long (nullable = true)

+---------+-------+------+
|dept_name|dept_id|salary|
+---------+-------+------+
|Finance  |10     |3000  |
|Marketing|20     |2000  |
|Sales    |30     |5000  |
|IT       |40     |6000  |
+---------+-------+------+



In [ ]:
deptDF.columns

['dept_name', 'dept_id', 'salary']

In [ ]:
def joinsDemo(df1,df2,jointype):

  if jointype in("full","leftanti"):

    d_out=df1.join(df2,df1.dept_id == df2.dept_id,jointype)
    d_out.filter("gender == 'M'").show()

  elif jointype == "fullouter":
    d_out=df1.join(df2,df1.dept_id == df2.dept_id,jointype)
    d_out.filter(df2.dept_id > 30).show()

  elif jointype == "inner":

    d_out=df1.join(df2,df1.salary > df2.salary,jointype)
    d_out.filter(df1.name.startswith("S")).show()
    #d_out.filter(df1.name.endswith("e")).show()

  elif jointype == "right":
    d_out=df1.join(df2,df1.dept_id == df2.dept_id,jointype)
    #d_out.select("emp_id","name").show()
    #d_out[d_out.name.isin("Smith","Rose")].show()
    #d_out.select("name","emp_id",d_out.name.like("%S%")).show()
    d_out.select(d_out.name.substr(1,3).alias("shortname")).show()

  else:

    d_out=df1.join(df2,df1.dept_id == df2.dept_id,jointype).select(df1.emp_id,df1.name,df1.gender,df1.salary,df2.dept_name,df2.dept_id)
    d_out=d_out.replace("","null")
    #d_out.select(d_out.columns[ 2: ])
    d_out.dropna().show()
    

  return d_out 

In [ ]:
joinsDemo(empDF,deptDF,"right")

+---------+
|shortname|
+---------+
|      Jon|
|      Wil|
|      Smi|
|      Ros|
|     null|
|      Bro|
+---------+



DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, dept_id: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint, salary: bigint]

In [ ]:
joinsDemo(empDF,deptDF,"inner")

+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|salary|
+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+
|     1|Smith|             -1|       2018|         10|     M|  3000|Marketing|     20|  2000|
+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+



DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint, salary: bigint]

In [ ]:
joinsDemo(empDF,deptDF,"full")

+------+--------+---------------+-----------+-----------+------+------+---------+-------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|salary|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|  2000|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+------+



DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint, salary: bigint]

In [ ]:
joinsDemo(empDF,deptDF,"leftanti")

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



In [ ]:
joinsDemo(empDF,deptDF,"outer")

+------+--------+------+------+---------+-------+
|emp_id|    name|gender|salary|dept_name|dept_id|
+------+--------+------+------+---------+-------+
|     1|   Smith|     M|  3000|  Finance|     10|
|     3|Williams|     M|  1000|  Finance|     10|
|     4|   Jones|     F|  2000|  Finance|     10|
|     2|    Rose|     M|  4000|Marketing|     20|
|     5|   Brown|  null|    -1|       IT|     40|
+------+--------+------+------+---------+-------+



DataFrame[emp_id: bigint, name: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint]

In [ ]:
joinsDemo(empDF,deptDF,"fullouter")

+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|salary|
+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+
|     5|Brown|              2|       2010|         40|      |    -1|       IT|     40|  6000|
+------+-----+---------------+-----------+-----------+------+------+---------+-------+------+



DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint, salary: bigint]

In [ ]:
joinsDemo(empDF,deptDF,"outer")

+------+--------+------+------+---------+-------+
|emp_id|    name|gender|salary|dept_name|dept_id|
+------+--------+------+------+---------+-------+
|     1|   Smith|     M|  3000|  Finance|     10|
|     3|Williams|     M|  1000|  Finance|     10|
|     4|   Jones|     F|  2000|  Finance|     10|
|     2|    Rose|     M|  4000|Marketing|     20|
|     5|   Brown|  null|    -1|       IT|     40|
+------+--------+------+------+---------+-------+



DataFrame[emp_id: bigint, name: string, gender: string, salary: bigint, dept_name: string, dept_id: bigint]

In [ ]:
def addPrefix(df1,prefix):
	for i in df1.columns:
		df1=df1.withColumnRenamed(i,'{}{}'.format(prefix,i))

	return df1

In [ ]:
addPrefix(empDF,1 )

DataFrame[emp_id1: bigint, name1: string, superior_emp_id1: bigint, year_joined1: string, emp_dept_id1: string, gender1: string, salary1: bigint]

In [ ]:
empDF.select(empDF.columns[2:4]).show(3)

+---------------+-----------+
|superior_emp_id|year_joined|
+---------------+-----------+
|             -1|       2018|
|              1|       2010|
|              1|       2010|
+---------------+-----------+
only showing top 3 rows



In [ ]:
"""def checkColumns(df1,column):
	for i in df1.columns:
		df1=df1.columns == 'null'

	return df1"""

In [ ]:
nan_cols=[i for i in empDF.columns if empDF[i].notna().any()]

TypeError: ignored

In [ ]:
print(len(empDF.columns))

7


In [ ]:
result=empDF.isEmpty
print(result)

<bound method DataFrame.isEmpty of DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint]>


In [ ]:
print(empDF)
is_nan=empDF.na
print(is_nan)

DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint]


In [ ]:
student=[(1,'noori','zafar'),(2,'neelu','khan'),(3,'afrin','begam')]
studentColumns=['id','fname','lname']

In [ ]:
studentDF=spark.createDataFrame(student,studentColumns)
studentDF.show()

+---+-----+-----+
| id|fname|lname|
+---+-----+-----+
|  1|noori|zafar|
|  2|neelu| khan|
|  3|afrin|begam|
+---+-----+-----+



In [ ]:
studentcourse=[(1,2),(1,3),(2,1),(2,2),(2,3),(3,1)]
studentcColumns=['studentid','courseid']
studentcourseDF=spark.createDataFrame(studentcourse,studentcColumns)
studentcourseDF.show()

+---------+--------+
|studentid|courseid|
+---------+--------+
|        1|       2|
|        1|       3|
|        2|       1|
|        2|       2|
|        2|       3|
|        3|       1|
+---------+--------+



In [ ]:
course=[(1,'database',1),(2,'english',2),(3,'python',1)]
courseColumns=['id','name','teacherid']
courseDF=spark.createDataFrame(course,courseColumns)
courseDF.show()

+---+--------+---------+
| id|    name|teacherid|
+---+--------+---------+
|  1|database|        1|
|  2| english|        2|
|  3|  python|        1|
+---+--------+---------+



In [ ]:
df1=studentDF.join(studentcourseDF)

In [ ]:
df1.join(courseDF).show(40,truncate=40)

+---------+-----+-----+---------+--------+---+--------+---------+
|studentid|fname|lname|studentid|courseid| id|    name|teacherid|
+---------+-----+-----+---------+--------+---+--------+---------+
|        1|noori|zafar|        1|       2|  1|database|        1|
|        1|noori|zafar|        1|       3|  1|database|        1|
|        1|noori|zafar|        2|       1|  1|database|        1|
|        1|noori|zafar|        1|       2|  2| english|        2|
|        1|noori|zafar|        1|       2|  3|  python|        1|
|        1|noori|zafar|        1|       3|  2| english|        2|
|        1|noori|zafar|        1|       3|  3|  python|        1|
|        1|noori|zafar|        2|       1|  2| english|        2|
|        1|noori|zafar|        2|       1|  3|  python|        1|
|        1|noori|zafar|        2|       2|  1|database|        1|
|        1|noori|zafar|        2|       3|  1|database|        1|
|        1|noori|zafar|        3|       1|  1|database|        1|
|        1

In [ ]:
df1.join(courseDF).show(2,truncate=False)

+---------+-----+-----+---------+--------+--------+--------+---------+
|studentid|fname|lname|studentid|courseid|courseid|name    |teacherid|
+---------+-----+-----+---------+--------+--------+--------+---------+
|1        |noori|zafar|1        |2       |1       |database|1        |
|1        |noori|zafar|1        |3       |1       |database|1        |
+---------+-----+-----+---------+--------+--------+--------+---------+
only showing top 2 rows



In [ ]:
def joinThree(df1,df2,df3,jointype1,jointype2):
  if jointype1 == "right":
    dout=df1.join(df2,df1.id==df2.studentid,jointype1)
    if jointype2 in("left","right"):
      dout=dout.join(df3,dout.courseid==df3.courseid,jointype2)
  elif jointype1 == "left":
    dout=df1.join(df2,df1.id==df2.studentid,jointype1)
    if jointype2 == "right":
      dout=dout.join(df3,dout.studentid==df3.id,jointype2)
    else:
      dout=dout.join(df3,dout.id ==df3.teacherid,jointype2)

  return dout

In [ ]:
joinThree(studentDF,studentcourseDF,courseDF,'left','full').show()

+---+-----+-----+---------+--------+----+--------+---------+
| id|fname|lname|studentid|courseid|  id|    name|teacherid|
+---+-----+-----+---------+--------+----+--------+---------+
|  1|noori|zafar|        1|       3|   1|database|        1|
|  1|noori|zafar|        1|       3|   3|  python|        1|
|  1|noori|zafar|        1|       2|   1|database|        1|
|  1|noori|zafar|        1|       2|   3|  python|        1|
|  2|neelu| khan|        2|       3|   2| english|        2|
|  2|neelu| khan|        2|       2|   2| english|        2|
|  2|neelu| khan|        2|       1|   2| english|        2|
|  3|afrin|begam|        3|       1|null|    null|     null|
+---+-----+-----+---------+--------+----+--------+---------+



In [ ]:
joinThree(studentDF,studentcourseDF,courseDF,'left','right').show()

+---+-----+-----+---------+--------+---+--------+---------+
| id|fname|lname|studentid|courseid| id|    name|teacherid|
+---+-----+-----+---------+--------+---+--------+---------+
|  1|noori|zafar|        1|       3|  1|database|        1|
|  1|noori|zafar|        1|       2|  1|database|        1|
|  3|afrin|begam|        3|       1|  3|  python|        1|
|  2|neelu| khan|        2|       3|  2| english|        2|
|  2|neelu| khan|        2|       2|  2| english|        2|
|  2|neelu| khan|        2|       1|  2| english|        2|
+---+-----+-----+---------+--------+---+--------+---------+

